Представляю базовое исследование контеста от сбербанка "Russian Housing Market". Сначала я проанализирую предоставленные данные, сделаю выводы, затем проведу предобработку данных, в конце построю baseline модель

Загрузим нужные библиотеки, загрузим файл, посмотрим сколько в нем строк и столбцов:

In [342]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

train = pd.read_csv("../input/train_without_noise.csv", parse_dates=['timestamp'])
print ("lines:", str(train.shape[0]) + ", " "cols:", train.shape[1])

Бросим взгляд на наши данные

In [343]:
train.tail(2).transpose()

Посмотрим на столбцы с промущенными значениями:

In [344]:
MISS = {}
for i in range(train.shape[1]):
    MISS[train.columns[i]] = sum(pd.isnull(train[train.columns[i]]))/len(train)
MISS = {k: v for k, v in MISS.items() if v > 0}
MISS = sorted(MISS.items(), key=lambda x: x[1], reverse=True )
hist_x = []
hist_y = []
for i in MISS:
    hist_x.append(i[0])
    hist_y.append(i[1])
df = pd.DataFrame({"x":hist_x, "y":hist_y})
df.columns = ['name', '% NULL']
ind = np.arange(df.shape[0])
width = 0.6
fig, lx = plt.subplots(figsize=(12,12))
rects = lx.barh(ind, df['% NULL'], color='orange')
lx.set_yticks(ind)
lx.set_yticklabels(df['name'], rotation='horizontal')
lx.set_xlabel("Пропущено значений")
plt.show()

Всего 52 колонки с пропущенными значениями,  максимально пропущено 45% значений.
Позднее надо будет принять решение что с этим делать - заполнить пропуски средними/медианными значениями, не заполнять, не брать переменные где много пропусков.

Преобразуем наши временные данные: разложим timestamp на день/месяц/год/сколько всего дней прошло с 1й транзакции, после удалим timestamp.

In [345]:
train["month"] = train["timestamp"].dt.month
train["year"] = train["timestamp"].dt.year
train["day"] = train["timestamp"].dt.day
train["days_passed"] = (train["timestamp"] - min(train["timestamp"])).apply(lambda x: x.days)

train.drop("timestamp", axis=1, inplace = True)

Посмотрим на распределение целевой переменной price_doc:

In [346]:
train['price_doc'].hist(bins=100)

Если не учитывать выбросы, оно в целом нормальное: (сразу замечаем выброс дешевых квартир стоимостью около 1 млн). Как выяснилось в датасете присутствовали фиктивные сделки по таким вот ценам с целью ухода от налогооблажения.

In [347]:
train['price_doc'][(train['price_doc'] >1000000) & (train['price_doc'] <20000000)].hist(bins=100)

В модели нам не нужны id квартир, убираем. За y берем логарифм цены (согласно метрике). Преобразовываем категориальные факторы в числа:

In [348]:
x = train.drop(["price_doc", "id"], axis=1)
y = np.log1p(train["price_doc"])

for col in x.columns:
    if x[col].dtype == 'object':
        lab = LabelEncoder()
        lab.fit(list(x[col].values)) 
        x[col] = lab.transform(list(x[col].values))

Разбираем наши данные на test и valid для локальной валидации, 80%-20%.
Задаем стартовые параметры для xgb, в дальнейшем будем их оптимизировать.  Строим кросс-валидацию. В данном случае мы используем кросс-валидацию для нахождения оптимального количества раундов дабы не переобучиться.

In [349]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2)

xgb_params = {'max_depth': 5,
              'eta': 0.1,             
              'objective': 'reg:linear',
              'eval_metric': 'rmse',
              'subsample': 0.8
}

In [350]:
mx_train = xgb.DMatrix(x_train, y_train)
mx_val = xgb.DMatrix(x_val, y_val)
full_train = xgb.DMatrix(x, y)

xgb_cv = xgb.cv(xgb_params, mx_train, num_boost_round=1000, early_stopping_rounds=5,
    verbose_eval=5, show_stdv=False)
xgb_cv[['train-rmse-mean', 'test-rmse-mean']].plot()

Модель построена, метрика на test около 0.4, на train 0.47. Пора подставить 20% наших отложенных данных для валидации. Наша модель довольно стабильна.

In [351]:
best_round = len(xgb_cv)
part_model = xgb.train(xgb_params, mx_train, num_boost_round= best_round)

preds = part_model.predict(mx_val)
validation =  {x : y for x in list(y_val) for y in  preds}

RMSE = 0
for x,y in validation.items():
    RMSE += (x - y)**2
RMSE = RMSE/len(validation)
RMSE

Теперь строим модель на всех данных, смотрим на значимость наших факторов.

In [352]:
full_model = xgb.train(xgb_params, full_train, num_boost_round= best_round)

fig, ax = plt.subplots(1, 1, figsize=(12, 15))
xgb.plot_importance(full_model, max_num_features=60, height=0.5, ax=ax)

Построим корреляционную матрицу топ факторов:

In [353]:
imp = full_model.get_fscore()
imp = sorted(imp.items(), key=lambda x: x[1], reverse=True )
imp = [x[0] for x in imp[:19]]
imp.append("price_doc")

fig, ax = plt.subplots(1, 1, figsize=(16, 16))
corr = train[imp].corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

Выводы:
* Наиболее важными факторы для предсказания цены продажи квартиры в данном соревновании являются: общая площадь квартиры, дней прошло, этаж, год постройки здания, жилая площадь, количество этажей в доме, растояние до жд и метро.
* В части значимых факторов есть пропуски (всего этажей в доме, год постройки) их нужно устранить в первую очередь.
* Нужно устранить выбросы в данных, в первую очередь слишком дорогие и слишком дешевые квартиры.
* Настройка гипер параметров модели. Наверняка уменьшив eta и увеличив кол-во раундов модель выиграет в точности.
* Для улучшения стабильности и прогнозных свойств стоит построить ансамбль моделей.
* Стоит нагенерировать еще факторов. Например соотношения жилая площадь/общая/кухня, этаж/максимальный этаж, год продажи/год постройки, географические расстояние до значимых объектов, макроэкономические факторы (инфляция, безработица, курс валюты)